In [1]:
from sklearn.datasets import fetch_20newsgroups
#Choosing categories we require 
categories = ['alt.atheism','soc.religion.christian','comp.graphics','sci.med']
#Will take data from the folder train/test
news_train = fetch_20newsgroups(subset='train',categories=categories,shuffle=True)
news_test = fetch_20newsgroups(subset='test',categories=categories,shuffle=True)

In [2]:
import nltk
nltk.download()
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [3]:
stemmer = SnowballStemmer("english", ignore_stopwords=True)

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])
    
stemmed_count_vect = StemmedCountVectorizer(stop_words='english')

text_mnb_stemmed = Pipeline([('vect', stemmed_count_vect),
                             ('tfidf', TfidfTransformer()),
                             ('mnb', GradientBoostingClassifier(random_state=10)),])

text_mnb_stemmed = text_mnb_stemmed.fit(news_train.data, news_train.target)

predicted_mnb_stemmed = text_mnb_stemmed.predict(news_test.data)

np.mean(predicted_mnb_stemmed == news_test.target)

0.8988015978695073

In [6]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
print("Accuracy:",accuracy_score(news_test.target,predicted_mnb_stemmed))
print(metrics.classification_report(news_test.target,predicted_mnb_stemmed,target_names=news_test.target_names)),metrics.confusion_matrix(news_test.target,predicted_mnb_stemmed)

Accuracy: 0.8894806924101198
                        precision    recall  f1-score   support

           alt.atheism       0.95      0.80      0.87       319
         comp.graphics       0.87      0.93      0.90       389
               sci.med       0.85      0.89      0.87       396
soc.religion.christian       0.91      0.92      0.92       398

              accuracy                           0.89      1502
             macro avg       0.90      0.89      0.89      1502
          weighted avg       0.89      0.89      0.89      1502



(None,
 array([[254,   9,  25,  31],
        [  1, 361,  26,   1],
        [  2,  36, 353,   5],
        [  9,   8,  13, 368]], dtype=int64))